In [1]:
# code source
# https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/tf2_arbitrary_image_stylization.ipynb#scrollTo=oXlcl8lqBgAD

import sys
import os
import random
from google.colab import drive
import functools
import json
import time


from matplotlib import gridspec
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

#drive.mount('/content/drive')
NST_path = "/content/drive/MyDrive/NST/"

def listdir_nohidden(path):
    listed = []
    for f in os.listdir(path):
        if not f.startswith('.'):
            listed.append(f)
    return listed

def crop_center(image):
  """Returns a cropped square image."""
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
  return image

@functools.lru_cache(maxsize=None)
def load_image(image_url, image_size=(256, 256), preserve_aspect_ratio=True):
  """Loads and preprocesses images."""
  # Cache image file locally.
  # image_path = tf.keras.utils.get_file(os.path.basename(image_url)[-128:], image_url)
  image_path = image_url
  # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
  img = tf.io.decode_image(
      tf.io.read_file(image_path),
      channels=3, dtype=tf.float32)[tf.newaxis, ...]
  img = crop_center(img)
  img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  return img

def show_n(images, titles=('',), name='a'):
  n = len(images)
  image_sizes = [image.shape[1] for image in images]
  w = (image_sizes[0] * 6) // 320
  plt.figure(figsize=(w * n, w))
  gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
  for i in range(n):
    plt.subplot(gs[i])
    plt.imshow(images[i][0], aspect='equal')
    plt.axis('off')
    plt.title(titles[i] if len(titles) > i else '')
  plt.gcf().savefig("/content/drive/MyDrive/NST/arbitrary/output/"+name+".jpg")
  
  

hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(hub_handle)

style_names = listdir_nohidden(NST_path+"dataset/style")
content_names = listdir_nohidden(NST_path+"dataset/content")

In [ ]:
%%time


content_img_size = (512, 512)
style_img_size = (256, 256)  # Recommended to keep it at 256.

c_name = random.choice(content_names)
s_name = random.choice(style_names)
#c_name = 'content14.jpg'
#s_name = 'witkacy2.jpg'

c_pairs = ["content14.jpg", "content16.jpg", "content22.jpg", "content25.jpg", "content27.jpg"]
s_pairs = ["witkacy2.jpg", "matejko5.jpg", "beksinski1.jpg", "wyspianski1.jpg", "witkacy5.jpg"]

for i in range(1):
    #print(str(i)+"/"+str(900))
    c_name = c_pairs[i]
    s_name = s_pairs[i]
    c_path = NST_path +"dataset/content/"+ c_name
    s_path = NST_path +"dataset/style/"+ s_name
    print(c_name, s_name)

    content_image = load_image(c_path, content_img_size)
    style_image = load_image(s_path, style_img_size)
    style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')

    start_time = time.time()
    stylized_image = hub_module(tf.constant(content_image), tf.constant(style_image))[0] #
    t={'total_time':str(time.time() - start_time)}
    with open('/content/drive/MyDrive/NST/arbitrary/output/'+c_name+"x"+s_name+".json", 'w') as fp:
        json.dump(t, fp)

    show_n([stylized_image],[c_name+"x"+s_name], name=c_name+"x"+s_name)
    show_n([content_image, style_image, stylized_image], titles=['Original content image', 'Style image', 'Stylized image'], name=c_name+"x"+s_name+"_all")

content14.jpg witkacy2.jpg


### todos
- zapisać 
  - czas
  - obraz potrójny
  - obraz stylizowany
- przeprowadzić na całym datasecie

### notes
- no GPU!
- styl portretu na krajobrazie - masakra

In [ ]:
from google.colab import runtime
runtime.unassign()